In [1]:
import sys
sys.path.append('../')

import torch
from networks.ResNet import ResNetMel
from torchsummary import summary
import torchvision.models as models

# To allow auto reload to this notebook after modifying any external file imported
%load_ext autoreload
%autoreload 2

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
resnet_mel = ResNetMel(num_classes=2).to(device)
print(resnet_mel)

/mnt/c/Users/abdal/Documents/Master/EMJMD MAIA/SEMESTER 3 - UdG/CAD/skin-lesion-dl-classifier/env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/mnt/c/Users/abdal/Documents/Master/EMJMD MAIA/SEMESTER 3 - UdG/CAD/skin-lesion-dl-classifier/env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNetMel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          

In [4]:
summary(resnet_mel, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [5]:
# Loop over all layers and check if they require gradients
for name, param in resnet_mel.named_parameters():
    print(f"Layer: {name}, Requires Grad: {param.requires_grad}")


Layer: model.conv1.weight, Requires Grad: False
Layer: model.bn1.weight, Requires Grad: False
Layer: model.bn1.bias, Requires Grad: False
Layer: model.layer1.0.conv1.weight, Requires Grad: False
Layer: model.layer1.0.bn1.weight, Requires Grad: False
Layer: model.layer1.0.bn1.bias, Requires Grad: False
Layer: model.layer1.0.conv2.weight, Requires Grad: False
Layer: model.layer1.0.bn2.weight, Requires Grad: False
Layer: model.layer1.0.bn2.bias, Requires Grad: False
Layer: model.layer1.0.conv3.weight, Requires Grad: False
Layer: model.layer1.0.bn3.weight, Requires Grad: False
Layer: model.layer1.0.bn3.bias, Requires Grad: False
Layer: model.layer1.0.downsample.0.weight, Requires Grad: False
Layer: model.layer1.0.downsample.1.weight, Requires Grad: False
Layer: model.layer1.0.downsample.1.bias, Requires Grad: False
Layer: model.layer1.1.conv1.weight, Requires Grad: False
Layer: model.layer1.1.bn1.weight, Requires Grad: False
Layer: model.layer1.1.bn1.bias, Requires Grad: False
Layer: model

In [15]:
# densnet
densenet_model = models.densenet161(pretrained=True)
print(densenet_model)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [16]:
densenet_model.classifier

Linear(in_features=2208, out_features=1000, bias=True)

In [53]:
from networks.DenseNet import DenseNetMel

In [54]:
densenet_mel = DenseNetMel(num_classes=2)

In [55]:
print(densenet_mel)

DenseNetMel(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (

In [56]:
# Loop over all layers and check if they require gradients
for name, param in densenet_mel.named_parameters():
    print(f"Layer: {name}, Requires Grad: {param.requires_grad}")


Layer: features.conv0.weight, Requires Grad: False
Layer: features.norm0.weight, Requires Grad: False
Layer: features.norm0.bias, Requires Grad: False
Layer: features.denseblock1.denselayer1.norm1.weight, Requires Grad: False
Layer: features.denseblock1.denselayer1.norm1.bias, Requires Grad: False
Layer: features.denseblock1.denselayer1.conv1.weight, Requires Grad: False
Layer: features.denseblock1.denselayer1.norm2.weight, Requires Grad: False
Layer: features.denseblock1.denselayer1.norm2.bias, Requires Grad: False
Layer: features.denseblock1.denselayer1.conv2.weight, Requires Grad: False
Layer: features.denseblock1.denselayer2.norm1.weight, Requires Grad: False
Layer: features.denseblock1.denselayer2.norm1.bias, Requires Grad: False
Layer: features.denseblock1.denselayer2.conv1.weight, Requires Grad: False
Layer: features.denseblock1.denselayer2.norm2.weight, Requires Grad: False
Layer: features.denseblock1.denselayer2.norm2.bias, Requires Grad: False
Layer: features.denseblock1.dens